In [1]:
from pprint import pprint

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
df_clients = pd.read_csv('clients.csv', index_col='client_id')
df_train = pd.read_csv('uplift_train.csv', index_col='client_id')
df_test = pd.read_csv('uplift_test.csv', index_col='client_id')


df_features = df_clients.copy()
df_features['first_issue_time'] = \
    (pd.to_datetime(df_features['first_issue_date'])
     - pd.to_datetime(df_features['first_issue_date']).min()) / pd.Timedelta('365d')

df_features['first_redeem_time'] = \
    (pd.to_datetime(df_features['first_redeem_date'])
     - pd.to_datetime(df_features['first_redeem_date']).min()) / pd.Timedelta('365d')

df_features['issue_redeem_delay'] = df_features['first_redeem_time'] \
    - df_features['first_issue_time']

df_features = df_features.join(pd.get_dummies(df_features['gender']))
df_features['first_redeem_time'] = df_features['first_redeem_time'].fillna(df_features['first_redeem_time'].mean())
df_features['issue_redeem_delay'] = df_features['issue_redeem_delay'].fillna(df_features['issue_redeem_delay'].mean())

df_features = df_features.drop(['first_issue_date', 'first_redeem_date', 'gender'], axis=1)

indices_train = df_train.index
indices_test = df_test.index
indices_learn, indices_valid = train_test_split(df_train.index, test_size=0.3, random_state=123)


X_train = df_features.loc[indices_learn, :]
y_train = df_train.loc[indices_learn, 'target']
treat_train = df_train.loc[indices_learn, 'treatment_flg']

X_val = df_features.loc[indices_valid, :]
y_val = df_train.loc[indices_valid, 'target']
treat_val =  df_train.loc[indices_valid, 'treatment_flg']

X_train_full = df_features.loc[indices_train, :]
y_train_full = df_train.loc[:, 'target']
treat_train_full = df_train.loc[:, 'treatment_flg']

X_test = df_features.loc[indices_test, :]

cat_features = ['gender']

In [2]:
import re
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
def build_base_network(input_shape):
    
    seq = Sequential()

    #flatten 
    seq.add(Dense(128, activation='relu'))
    seq.add(Dense(128, activation='relu'))
    seq.add(Dense(50, activation='relu'))
    return seq

In [4]:
input_dim = (8, )
input_a = Input(shape=input_dim)
input_b = Input(shape=input_dim)

base_network = build_base_network(input_dim)
feat_vecs_a = base_network(input_a)
feat_vecs_b = base_network(input_b)

In [5]:
# def output_uplift(vects):
#     m0, m1 = vects
    
#     return m0, m1


# def uplift_output_shape(shapes):
#     shape1, shape2 = shapes
#     return (shape1[0], 2)

# uplift = Lambda(output_uplift, output_shape=uplift_output_shape)([feat_vecs_a, feat_vecs_b])

In [6]:
epochs = 2
rms = RMSprop()
output = Dense(1, activation='sigmoid')(feat_vecs_a)

model = Model(input=[input_a, input_b], output=[output, output])

In [10]:
def smite_loss(y_true, y_pred):
    m0, m1 = y_pred[0], y_pred[1]
    t = treat_train
    mt = K.dot(t, m1) + K.dot((1 - t), m0)
    z = [FlaggedFloat(t * o / 0.5 - (1 - t) * o / 0.5) for o, t in zip(y_true, t)]
    J = K.mean(K.square(z - (m1 - m0)))
    L = K.sum(y_true * K.log(mt) + (1 - y_true) * K.log(1 - mt))
    return (1 - 0.2) * J + 0.2 * L 

model.compile(loss=smite_loss, optimizer=rms)

Tensor("loss_3/dense_4_loss/smite_loss/strided_slice_1:0", shape=(), dtype=float32)


AttributeError: 'tuple' object has no attribute 'rank'

In [ ]:
sample_1 = X_train.assign(treatment = 1)
sample_2 = X_train.assign(treatment = 0) 

model.fit([sample_1, sample_2], y_train, validation_split=.25, batch_size=1, verbose=2, nb_epoch=epochs)

In [ ]:
model.predict([X_test.assign(treatment = 1), X_test.assign(treatment = 0)])